In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

RAW_DIR = Path(r"event_dataset/evs_raw")
APS_DIR = Path(r"mydata/test/scene1/aps_png")
OUT_DIR = Path(r"mydata/test/scene1/events_11.10_scene1")

EVS_INFO = Path(r"timestamp\scene1\normal_v2_816_612_20250703093040412_info.txt")
APS_INFO = Path(r"timestamp\scene1\quadbayer_10bit_3264_2448_20250703093040412_info.txt")

OUT_DIR.mkdir(parents=True, exist_ok=True)

H, W = 612, 816
print("Paths set. OUT_DIR =", OUT_DIR.resolve())

Paths set. OUT_DIR = D:\事件相机projects\process-event-dataset\mydata\test\scene1\events_11.10_scene1


In [ ]:

import os
from pathlib import Path

def rename_files_to_last_six_digits(directory: Path):
    """重命名目录中的所有文件，只保留后六位数字"""
    for file_path in directory.glob('*'):
        if file_path.is_file():
            stem = file_path.stem
            suffix = file_path.suffix
            
            # 提取后六位数字
            if len(stem) >= 6:
                last_six = stem[-6:]
                new_name = f"{last_six}{suffix}"
                new_path = directory / new_name
                
                # 重命名文件
                file_path.rename(new_path)
                print(f"重命名: {file_path.name} -> {new_name}")

# 重命名 downsampled_aps 文件夹中的文件
downsampled_dir = APS_DIR
rename_files_to_last_six_digits(downsampled_dir)


print("文件重命名完成")

重命名: 000000.png -> 000000.png
重命名: 000001.png -> 000001.png
重命名: 000002.png -> 000002.png
重命名: 000003.png -> 000003.png
重命名: 000004.png -> 000004.png
重命名: 000005.png -> 000005.png
重命名: 000006.png -> 000006.png
重命名: 000007.png -> 000007.png
重命名: 000008.png -> 000008.png
重命名: 000009.png -> 000009.png
重命名: 000010.png -> 000010.png
重命名: 000011.png -> 000011.png
重命名: 000012.png -> 000012.png
重命名: 000013.png -> 000013.png
重命名: 000014.png -> 000014.png
重命名: 000015.png -> 000015.png
重命名: 000016.png -> 000016.png
重命名: 000017.png -> 000017.png
重命名: 000018.png -> 000018.png
重命名: 000019.png -> 000019.png
重命名: 000020.png -> 000020.png
重命名: 000021.png -> 000021.png
重命名: 000022.png -> 000022.png
重命名: 000023.png -> 000023.png
重命名: 000024.png -> 000024.png
重命名: 000025.png -> 000025.png
重命名: 000026.png -> 000026.png
重命名: 000027.png -> 000027.png
重命名: 000028.png -> 000028.png
重命名: 000029.png -> 000029.png
重命名: 000030.png -> 000030.png
重命名: 000031.png -> 000031.png
重命名: 000032.png -> 000032.png
重命名: 00003

In [ ]:

def read_two_cols(path: Path):
    df = pd.read_csv(path, sep=None, engine="python", dtype=str)
    df.columns = [c.strip().lower() for c in df.columns]
    # 仅取 index, timestamp 两列
    sub = df[["index", "timestamp"]].copy()
    sub["index"] = sub["index"].astype(int)
    sub["timestamp"] = sub["timestamp"].astype(int)
    sub = sub.sort_values("index").reset_index(drop=True)
    return sub

aps_df = read_two_cols(APS_INFO)
evs_df = read_two_cols(EVS_INFO)

aps_ts = aps_df["timestamp"].to_numpy()   # 长度 200
evs_ts = evs_df["timestamp"].to_numpy()   # 长度 7800


aps_files = sorted(APS_DIR.glob("*.png"))
raw_files = sorted(RAW_DIR.glob("*.raw"))

print("Loaded timestamps and listed files:",
      f"APS frames={len(aps_files)}, EVS raws={len(raw_files)}")

Loaded timestamps and listed files: APS frames=200, EVS raws=7800


In [5]:
display(aps_df.head())
display(evs_df.head())

,index,timestamp
0,0,53857575
1,1,53907563
2,2,53957551
3,3,54007539
4,4,54057527


,index,timestamp
0,0,53843860
1,1,53845113
2,2,53846366
3,3,53847619
4,4,53848872


In [ ]:

H, W = 612, 816

# 256 长度 LUT：默认 0，其它仅设置 1 和 2

def decode_raw_to_events(raw_path: Path, t) -> np.ndarray:

    buf = np.frombuffer(raw_path.read_bytes(), dtype=np.uint8)
    buf = buf.copy()
    assert buf.size == H * W, f"{raw_path.name}: size={buf.size} != {H*W} (H*W)"
    pol = buf.reshape(H, W)  # 映射并 reshape
    pol[pol==1]= -1  # 将 1 映射为 -1
    pol[pol==2]= +1  # 将 2 映射为 +
    
    ys, xs = np.nonzero(pol)  # 仅保留非零事件
    if ys.size == 0:
        return np.empty((0, 4), dtype=np.int64)

    pvals = pol[ys, xs].astype(np.int8)
    ts = np.full(xs.shape, np.int64(t), dtype=np.int64)

    # 统一输出为 int64，列顺序 [x, y, t, p]
    events = np.stack([xs.astype(np.int64),
                       ys.astype(np.int64),
                       ts,
                       pvals.astype(np.int64)], axis=1)
    return events




In [7]:
def decode_raw_to_events_pol(raw_path: Path, t) -> np.ndarray:

    buf = np.frombuffer(raw_path.read_bytes(), dtype=np.uint8)
    buf = buf.copy()
    assert buf.size == H * W, f"{raw_path.name}: size={buf.size} != {H*W} (H*W)"
    pol = buf.reshape(H, W)  # 映射并 reshape
    pol[pol==1]= -1  # 将 1 映射为 -1
    pol[pol==2]= +1  # 将 2 映射为 +

    return pol

In [8]:
import cv2
def _event_to_image(event, path):
    h, w = event.shape
    print(f"W:{w},H:{h}")
    image = np.zeros((h, w, 3)) + 255
    image[event[:] > 0] = [0, 0, 255]
    image[event[:] < 0] = [255, 0, 0]
    cv2.imwrite(path, image)
demo_events = decode_raw_to_events_pol(raw_files[1200], evs_ts[1200])
_event_to_image(demo_events, r"1.png")

W:816,H:612


C:\Users\QoHN\AppData\Local\Temp\ipykernel_23044\1622215336.py:7: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  pol[pol==1]= -1  # 将 1 映射为 -1


In [9]:
# 试跑第一个 raw（对应第一个 EVS 时间戳）
demo_events = decode_raw_to_events(raw_files[0], evs_ts[0])
print("first RAW:", raw_files[0].name)
print("t =", int(evs_ts[0]))
print("events shape:", demo_events.shape)

# 前 10 行预览
import pandas as pd
print(pd.DataFrame(demo_events[:10], columns=["x","y","t","p"]).to_string(index=False)) #TimeLensXL
# print(pd.DataFrame(demo_events[:10], columns=["x","y","timestamp","polarity"]).to_string(index=False)) #CBMNet

# 极性分布看看
unique, counts = np.unique(demo_events[:,3], return_counts=True)
print("polarity counts:", dict(zip(unique.tolist(), counts.tolist())))


first RAW: 816_612_8_0000000000.raw
t = 53843860
events shape: (2791, 4)
  x  y        t  p
410  0 53843860 -1
514  0 53843860 -1
317  1 53843860 -1
410  1 53843860 -1
411  1 53843860 -1
550  1 53843860 -1
625  1 53843860  1
 17  2 53843860  1
189  2 53843860 -1
579  2 53843860 -1
polarity counts: {-1: 1975, 1: 816}


C:\Users\QoHN\AppData\Local\Temp\ipykernel_23044\2519770733.py:15: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  pol[pol==1]= -1  # 将 1 映射为 -1


In [ ]:
TOTAL_EVENTS = 0
written = []

for i in tqdm(range(199), desc="Aggregating by APS intervals"):
    t0 = aps_ts[i]
    t1 = aps_ts[i+1]
    
    mask = (evs_ts >= t0) & (evs_ts < t1)
    idxs = np.nonzero(mask)[0]
    
    parts = []
    for j in idxs:
        raw_path = raw_files[j] 
        t = evs_ts[j]
        ev = decode_raw_to_events(raw_path, t)
        if ev.size:
            parts.append(ev)
    
    if parts:
        events = np.concatenate(parts, axis=0)
    else:
        events = np.empty((0, 4), dtype=np.int64)
    
    TOTAL_EVENTS += events.shape[0]
    
    # 保存为与第 i 帧 PNG 对应的文件名（去扩展名 + _events.npy）
    stem = aps_files[i].stem
    out_path = OUT_DIR / f"{stem}.npy"
    np.save(out_path, events)
    written.append(out_path)
    
    print(f"[{i:03d}] EVS slices={len(idxs)}, events={events.shape[0]}, saved -> {out_path.name}")

print(f"Done. Files written: {len(written)}; total events: {TOTAL_EVENTS}")

Aggregating by APS intervals:   0%|          | 0/199 [00:00<?, ?it/s]

C:\Users\QoHN\AppData\Local\Temp\ipykernel_23044\2519770733.py:15: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  pol[pol==1]= -1  # 将 1 映射为 -1
Aggregating by APS intervals:   1%|          | 2/199 [00:00<00:16, 11.93it/s]

[000] EVS slices=39, events=124769, saved -> 000000.npy
[001] EVS slices=39, events=115899, saved -> 000001.npy
[002] EVS slices=39, events=120694, saved -> 000002.npy


Aggregating by APS intervals:   3%|▎         | 6/199 [00:00<00:17, 11.26it/s]

[003] EVS slices=39, events=130233, saved -> 000003.npy
[004] EVS slices=39, events=206923, saved -> 000004.npy
[005] EVS slices=39, events=237555, saved -> 000005.npy


...

[192] EVS slices=39, events=347035, saved -> 000192.npy
[193] EVS slices=39, events=348110, saved -> 000193.npy
[194] EVS slices=39, events=343750, saved -> 000194.npy


Aggregating by APS intervals:  99%|█████████▉| 198/199 [00:18<00:00, 10.33it/s]

[195] EVS slices=39, events=298863, saved -> 000195.npy
[196] EVS slices=39, events=273473, saved -> 000196.npy
[197] EVS slices=39, events=266639, saved -> 000197.npy


Aggregating by APS intervals: 100%|██████████| 199/199 [00:19<00:00, 10.44it/s]

[198] EVS slices=39, events=251177, saved -> 000198.npy
Done. Files written: 199; total events: 61637715


In [ ]:
import random

print("Expect 199 files; got:", len(list(OUT_DIR.glob("*.npy"))))

# 随机抽查 2 个文件
written_files = sorted(OUT_DIR.glob("*.npy"))
sample_n = min(2, len(written_files))
samples = random.sample(written_files, sample_n) if sample_n > 0 else []

for fp in samples:
    arr = np.load(fp, allow_pickle=False)
    print(fp.name, "shape:", arr.shape, "dtype:", arr.dtype)
    if arr.size > 0:
        unique_p = np.unique(arr[:, 3])
        print("p values:", unique_p.tolist())
    else:
        print("empty array.")
print("Check complete.")

Expect 199 files; got: 199
000143.npy shape: (315646, 4) dtype: int64
p values: [-1, 1]
000118.npy shape: (168390, 4) dtype: int64
p values: [-1, 1]
Check complete.


In [23]:
from pathlib import Path

# 原始APS帧目录
aps_dir = Path(r"mydata\test\scene1\aps_png")

# 下采样后输出目录
images_dir = Path(r"mydata\test\scene1\downsampled_aps")

# 目标分辨率
ev_w = 816
ev_h = 612

In [24]:
from PIL import Image

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def list_sorted_numeric(path: Path, suffix):
    files = sorted(path.glob(f"*{suffix}"))
    def _key(f):
        try:
            return int(f.stem)
        except ValueError:
            return f.stem
    return sorted(files, key=_key)

def downsample_images(aps_dir: Path, images_dir: Path, ev_w: int, ev_h: int):
    ensure_dir(images_dir)
    pngs = list_sorted_numeric(aps_dir, ".png")
    if not pngs:
        raise FileNotFoundError(f"No PNGs found in {aps_dir}")
    for p in pngs:
        im = Image.open(p).convert("RGB")
        im_ds = im.resize((ev_w, ev_h), resample=Image.BILINEAR)
        im_ds.save(images_dir / p.name)

In [27]:
# 设置路径和参数
aps_dir = Path("mydata\\test\\scene1\\aps_png")
images_dir = Path("mydata\\test\\scene1\\downsampled_aps")
ev_w, ev_h = 816, 612

# 执行下采样
downsample_images(aps_dir, images_dir, ev_w, ev_h)

In [ ]:
from pathlib import Path

# 原始APS帧目录
aps_dir = APS_DIR

# 下采样后输出目录
images_dir = Path(r"mydata\test\scene4\downsampled_aps")

# 目标分辨率
ev_w = 816
ev_h = 612

In [ ]:
from PIL import Image

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def list_sorted_numeric(path: Path, suffix):
    files = sorted(path.glob(f"*{suffix}"))
    def _key(f):
        try:
            return int(f.stem)
        except ValueError:
            return f.stem
    return sorted(files, key=_key)

def downsample_images(aps_dir: Path, images_dir: Path, ev_w: int, ev_h: int):
    ensure_dir(images_dir)
    pngs = list_sorted_numeric(aps_dir, ".png")
    if not pngs:
        raise FileNotFoundError(f"No PNGs found in {aps_dir}")
    for p in pngs:
        im = Image.open(p).convert("RGB")
        im_ds = im.resize((ev_w, ev_h), resample=Image.BILINEAR)
        im_ds.save(images_dir / p.name)

In [ ]:
# 执行下采样
downsample_images(aps_dir, images_dir, ev_w, ev_h)